In [1]:
from fiber_properties import (frd, image_list, FiberImage, save_plot,
                              plot_frd_encircled_energy,
                              plot_frd_encircled_energy_comparison,
                              plot_frd_input_output,
                              plot_frd_energy_loss)

NEW_OBJECTS = False
NEW_DATA = False
FOCAL_RATIO_DIAMETER = 0.95
FRD_CALIBRATION_THRESHOLD = 1500

print 'Complete'


Complete


In [2]:
# Helpful function and a container class

class Container(object):
    def __init__(self, name, folder, in_f, out_f):
        self.name = name
        self.folder = folder
        self.in_objs = image_objects(folder, 'input', in_f)
        self.out_objs = image_objects(folder, 'output', out_f)
        self.output = None
        self.magn = None
        self.magn_list = None
        self.magn_error = None

def ambient_files(folder):
    return image_list(folder+'ambient/ff_', num=3)

def image_file(folder, image_type='input', f=3.0):
    return folder+image_type+'_'+str(f)+'.fit'

def object_file(folder, image_type='input', f=3.0):
    return folder+image_type+'_'+str(f)+'_obj.pkl'

def image_objects(folder, image_type='input', f_list=[]):
    if NEW_OBJECTS:
        for f in f_list:
            print 'Saving ' + object_file(folder, image_type, f)
            im_obj = FiberImage(image_file(folder, image_type, f),
                                ambient=ambient_files(folder),
                                threshold=FRD_CALIBRATION_THRESHOLD,
                                camera='ff')
            setattr(im_obj._frd_info, image_type+'_fnum', f)
            im_obj.save_object(object_file(folder, image_type, f))
    return [object_file(folder, image_type, f) for f in f_list]

print 'Complete'


Complete


In [3]:
# Populate the data

in_f = [2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
out_f = [3.0, 4.0, 5.0]

TITLE = 'Fiber Mirror Octagonal'
FOLDER = '../data/EXPRES/fiber_mirror_oct/frd/'
TESTS = [Container(TITLE, FOLDER, in_f, out_f)]

for test in TESTS:
    print 'Calculating FRD for '+ test.name + ' Fiber'
    output = frd(test.in_objs, test.out_objs,
                 cal_method='full', save_objs=True,
                 fnum_diameter=FOCAL_RATIO_DIAMETER, new=NEW_DATA)
    test.output = output[0]
    test.magn = output[1]
    test.magn_list = output[2]
    test.magn_error = output[3]
    
    plot_frd_encircled_energy(output)
    save_plot(test.folder + test.name + ' FRD.png')

frd_outputs = [(test.output, test.magn, test.magn_list, test.magn_error) for test in TESTS]
labels = [test.name for test in TESTS]

print 'Complete'


Calculating FRD for Fiber Mirror Octagonal Fiber
../data/EXPRES/fiber_mirror_oct/frd/output_3.0_obj.pkl
../data/EXPRES/fiber_mirror_oct/frd/output_4.0_obj.pkl
../data/EXPRES/fiber_mirror_oct/frd/output_5.0_obj.pkl
../data/EXPRES/fiber_mirror_oct/frd/input_2.5_obj.pkl
../data/EXPRES/fiber_mirror_oct/frd/input_3.0_obj.pkl
../data/EXPRES/fiber_mirror_oct/frd/input_3.5_obj.pkl
../data/EXPRES/fiber_mirror_oct/frd/input_4.0_obj.pkl
../data/EXPRES/fiber_mirror_oct/frd/input_4.5_obj.pkl
../data/EXPRES/fiber_mirror_oct/frd/input_5.0_obj.pkl
Complete


In [4]:
plot_frd_energy_loss(frd_outputs, labels)
save_plot(FOLDER + 'Energy Loss.png')

plot_frd_input_output(frd_outputs, labels)
save_plot(FOLDER + 'Input vs Output.png')

plot_frd_encircled_energy_comparison(frd_outputs, labels)
save_plot(FOLDER + 'Encircled Energy Comparison.png')

print 'Complete'


Complete


In [5]:

def plot_frd_encircled_energy(frd_output):
    frd_info = frd_output[0]
    magn = frd_output[1]
    magn_list = frd_output[2]
    magn_error = frd_output[3]

    plt.figure()
    for i, f in enumerate(frd_info.input_fnum):
        plt.plot(frd_info.encircled_energy_fnum[i],
                 frd_info.encircled_energy[i],
                 label=str(f),
                 linewidth=2)
    plt.xlabel('output f-ratio')
    plt.ylabel('encircled energy')
    plt.ylim(ymax=1)
    plt.grid()
    plt.legend(loc=3, title='input f-ratio')

def plot_frd_energy_loss(frd_outputs, labels):
    plt.figure()

    for i, output in enumerate(frd_outputs):
        frd_info = output[0]
        magn = output[1]
        magn_list = output[2]
        magn_error = output[3]

        plt.errorbar(frd_info.input_fnum,
                     frd_info.energy_loss,
                     xerr=magn_error*np.array(frd_info.input_fnum),
                     label=labels[i],
                     linewidth=2)
    plt.xlabel('input f-ratio')
    plt.ylabel('energy loss (\%)')
    plt.grid()
    plt.legend(loc=2)

def plot_frd_input_output(frd_outputs, labels, ideal=True):
    plt.figure()

    for i, output in enumerate(frd_outputs):
        frd_info = output[0]
        magn = output[1]
        magn_list = output[2]
        magn_error = output[3]

        plt.errorbar(frd_info.input_fnum,
                     frd_info.output_fnum,
                     xerr=magn_error*np.array(frd_info.input_fnum),
                     yerr=magn_error*np.array(frd_info.input_fnum),
                     label=labels[i],
                     linewidth=2)

    if ideal:
        plt.plot(frd_info.input_fnum, frd_info.input_fnum,
                 label='Ideal', linestyle='--', color='black')
    plt.xlabel('input f-ratio')
    plt.ylabel('output f-ratio')
    plt.grid()
    plt.legend(loc=2)

def plot_frd_encircled_energy_comparison(frd_outputs, labels):
    plt.figure(figsize=[18,18])

    for i, output in enumerate(frd_outputs):
        frd_info = output[0]
        magn = output[1]
        magn_list = output[2]
        magn_error = output[3]

        for j, f in enumerate([2.5, 3.0, 3.5, 4.0, 4.5, 5.0]):
            if f in frd_info.input_fnum:
                plt.subplot(3, 2, j+1)
                index = frd_info.input_fnum.index(f)
                plt.plot(frd_info.encircled_energy_fnum[index],
                         frd_info.encircled_energy[index],
                         label=labels[i],
                         linewidth=2)
                plt.xlabel('output f-ratio')
                plt.ylabel('encircled energy')
                plt.ylim(ymax=1)
                plt.grid()
                plt.legend(title='input f/' + str(f), loc=3)

In [12]:
test = TESTS[0]
print test.output.encircled_energy[2][7]
print test.output.encircled_energy_fnum[2]

0.981604141643
[2.2999999999999998, 2.3999999999999999, 2.5, 2.6000000000000001, 2.7000000000000002, 2.8000000000000003, 2.9000000000000004, 3.0000000000000004, 3.1000000000000005, 3.2000000000000006, 3.3000000000000007, 3.4000000000000008, 3.5000000000000009, 3.600000000000001, 3.7000000000000011, 3.8000000000000012, 3.9000000000000012, 4.0000000000000018, 4.1000000000000014, 4.2000000000000011, 4.3000000000000016, 4.4000000000000021, 4.5000000000000018, 4.6000000000000014, 4.700000000000002, 4.8000000000000025, 4.9000000000000021, 5.0000000000000018, 5.1000000000000023, 5.2000000000000028, 5.3000000000000025, 5.4000000000000021, 5.5000000000000027, 5.6000000000000032, 5.7000000000000028, 5.8000000000000025, 5.900000000000003, 6.0000000000000036]
